In [1]:
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt
import os

In [2]:
def analyze_previous_vs_future_hits(db_path: str) -> pd.DataFrame:
    """
    Returns a DataFrame showing, for each actor:
    - How many earlier hits they had (>50M views)
    - How many future titles they appeared in
    - How many of those future titles were hits
    - Their future hit rate

    Only includes titles that have a release_year AND are linked through roles.
    """

    conn = sqlite3.connect(db_path)

    query = """
    SELECT
        actor_name,
        SUM(CASE WHEN future_views > 50000000 THEN 1 ELSE 0 END) AS num_future_hits,
        COUNT(*) AS num_future_titles,
        ROUND(
            1.0 * SUM(CASE WHEN future_views > 50000000 THEN 1 ELSE 0 END)
            / COUNT(*), 3
        ) AS future_hit_rate
    FROM (
        SELECT
            Actors.id,
            Actors.name AS actor_name,
            t1.views AS previous_hit_views,
            t2.views AS future_views
        FROM Roles r1
        JOIN Roles r2
            ON r1.actor_id = r2.actor_id
        JOIN Titles t1 ON r1.title_id = t1.id
        JOIN Titles t2 ON r2.title_id = t2.id
        JOIN Actors a ON Actors.id = r1.actor_id
        WHERE
            t1.release_year IS NOT NULL AND
            t2.release_year IS NOT NULL AND
            t1.release_year < t2.release_year AND
            t1.views > 50000000         -- previous title was a hit
    ) AS paired
    GROUP BY actor_name
    HAVING num_future_titles > 0       -- ensure future titles exist
    ORDER BY future_hit_rate DESC;
    """

    df = pd.read_sql_query(query, conn)
    conn.close()
    return df
